# WorkTree - Sistema de Análise de Compatibilidade com IA Generativa

**Global Solution - 2º Semestre FIAP**  
**Disciplina: DISRUPTIVE ARCHITECTURES: IOT, IOB & GENERATIVE IA**

---

## Desenvolvido por:
- **Felipe Seiki Hashiguti** - RM: 98985
- **Lucas Corradini Silveira** - RM: 555118  
- **Matheus Gregorio Mota** - RM: 557254

---

## Sobre o Projeto

Este notebook implementa um **sistema de análise de compatibilidade candidato-vaga** usando **IA Generativa (Google Gemini)** integrado à plataforma mobile **WorkTree**.

### Objetivo
Quando um candidato acessa uma vaga no app mobile, o sistema analisa automaticamente:
- Compatibilidade percentual entre candidato e vaga
- Habilidades em comum vs. habilidades faltantes
- Recomendações personalizadas de desenvolvimento
- Pontos fortes e áreas de melhoria

### Tecnologias Utilizadas
- **Google Gemini API** (IA Generativa)
- **Prompt Engineering** para análises precisas
- **Python** com bibliotecas de análise de dados
- **Flask REST API** para integração com app mobile

### Integração
A API REST desenvolvida aqui será consumida pelo app mobile **WorkTree** para exibir a análise de compatibilidade em tempo real ao candidato.

---

In [ ]:
# Importação de bibliotecas
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Any
import warnings

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Bibliotecas importadas com sucesso!")
print(f"📅 Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

## 💼 Parte 4: Banco de Dados de Vagas

Dados mockados de vagas baseados na estrutura do app WorkTree.

In [ ]:
# Configuração do modelo Gemini
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',  # SEM o prefixo 'models/' - CORRIGIDO
    generation_config={
        'temperature': 0.7,  # Criatividade controlada
        'top_p': 0.95,
        'top_k': 40,
        'max_output_tokens': 2048,  # Respostas detalhadas
    }
)

print("✅ Modelo Gemini 1.5 Flash configurado com sucesso!")
print("📊 Parâmetros:")
print("   - Temperature: 0.7 (análise balanceada)")
print("   - Max Tokens: 2048 (respostas completas)")
print("   - Top-p: 0.95 (alta qualidade)")


## 🤖 Parte 3: Configuração do Modelo Gemini

Configuração dos parâmetros do modelo de IA Generativa.

## 📚 Parte 2: Importação de Bibliotecas

Importação de todas as bibliotecas necessárias para o projeto.

In [ ]:
# Configuração da API Key do Google Gemini
# IMPORTANTE: Substitua pela sua própria API Key
# Obtenha gratuitamente em: https://makersuite.google.com/app/apikey

GOOGLE_API_KEY = "YOUR_API_KEY_HERE"  # ⚠️ SUBSTITUA PELA SUA API KEY

# Configurar a API
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

print("🔑 API Key configurada!")
print("⚠️  LEMBRE-SE: Nunca compartilhe sua API Key publicamente!")
print("⚠️  Para produção, use variáveis de ambiente (os.environ.get('GOOGLE_API_KEY'))")

## 🔑 Parte 1: Configuração da API Key do Google Gemini

**IMPORTANTE:** Configure sua API Key antes de executar as células seguintes.  
Obtenha gratuitamente em: https://makersuite.google.com/app/apikey

In [ ]:
# Estrutura de dados para Candidatos (baseado no app WorkTree)
candidates_database = [
    {
        "id": "c1",
        "name": "João Silva",
        "email": "joao@email.com",
        "title": "Desenvolvedor Full Stack",
        "experience_years": 6,
        "education": "Bacharelado em Ciência da Computação - USP",
        "bio": """Desenvolvedor Full Stack apaixonado por tecnologia, com sólida experiência 
        em desenvolvimento web e mobile. Busco desafios que me permitam crescer profissionalmente 
        e contribuir com soluções inovadoras.""",
        "skills": [
            "Node.js", "TypeScript", "JavaScript",
            "Python", "SQL", "MongoDB", "Docker",
            "Git", "REST API", "Microservices", "AWS"
        ],
        "certifications": [
            "AWS Certified Developer",
            "Python Advanced Certification"
        ],
        "languages": ["Português (Nativo)", "Inglês (Avançado)"],
        "previous_roles": [
            {
                "title": "Desenvolvedor Full Stack",
                "company": "Tech Corp",
                "duration": "3 anos",
                "description": "Desenvolvimento de aplicações web escaláveis"
            },
            {
                "title": "Desenvolvedor Backend",
                "company": "StartUp Digital",
                "duration": "2 anos",
                "description": "Desenvolvimento de APIs e microserviços"
            }
        ]
    },
    {
        "id": "c2",
        "name": "Maria Santos",
        "email": "maria@email.com",
        "title": "Desenvolvedora Mobile",
        "experience_years": 3,
        "education": "Tecnólogo em Sistemas para Internet - FIAP",
        "bio": """Desenvolvedora mobile com experiência em criar aplicativos para iOS e Android. 
        Adoro trabalhar em equipe e aprender novas tecnologias.""",
        "skills": [
            "JavaScript", "TypeScript",
            "Mobile Development", "Firebase", "Git",
            "REST API", "UI/UX Design"
        ],
        "certifications": [
            "Mobile Development Specialist"
        ],
        "languages": ["Português (Nativo)", "Inglês (Intermediário)"],
        "previous_roles": [
            {
                "title": "Desenvolvedora Mobile",
                "company": "AppTech Solutions",
                "duration": "2 anos",
                "description": "Desenvolvimento de apps para iOS e Android"
            },
            {
                "title": "Desenvolvedora Junior",
                "company": "Digital Agency",
                "duration": "1 ano",
                "description": "Desenvolvimento web e mobile"
            }
        ]
    },
    {
        "id": "c3",
        "name": "Pedro Costa",
        "email": "pedro@email.com",
        "title": "Analista de Dados",
        "experience_years": 2,
        "education": "Bacharelado em Estatística - UNICAMP",
        "bio": """Analista de dados entusiasta, com forte base em estatística e programação. 
        Busco transformar dados em insights valiosos para o negócio.""",
        "skills": [
            "Python", "Pandas", "NumPy", "Scikit-learn",
            "SQL", "Excel", "Statistics", "Data Visualization",
            "Matplotlib", "Seaborn"
        ],
        "certifications": [
            "Google Data Analytics Professional Certificate"
        ],
        "languages": ["Português (Nativo)", "Inglês (Básico)"],
        "previous_roles": [
            {
                "title": "Analista de Dados Júnior",
                "company": "Consulting Firm",
                "duration": "1.5 anos",
                "description": "Análise de dados e criação de relatórios"
            },
            {
                "title": "Estagiário de BI",
                "company": "Retail Company",
                "duration": "1 ano",
                "description": "Suporte em análises de vendas"
            }
        ]
    },
    {
        "id": "c4",
        "name": "Ana Oliveira",
        "email": "ana@email.com",
        "title": "Desenvolvedora Front-end",
        "experience_years": 4,
        "education": "Bacharelado em Sistemas de Informação - FIAP",
        "bio": """Desenvolvedora front-end criativa, focada em criar interfaces elegantes e 
        funcionais. Experiência com frameworks modernos e design systems.""",
        "skills": [
            "JavaScript", "TypeScript", "HTML", "CSS",
            "Styled Components", "Git", "Figma",
            "REST API"
        ],
        "certifications": [],
        "languages": ["Português (Nativo)", "Inglês (Básico)"],
        "previous_roles": [
            {
                "title": "Desenvolvedora Front-end",
                "company": "Web Agency",
                "duration": "3 anos",
                "description": "Desenvolvimento de sites e aplicações web"
            }
        ]
    }
]

print(f"{len(candidates_database)} candidatos carregados com sucesso!")
for candidate in candidates_database:
    print(f"   - {candidate['name']} - {candidate['title']} ({candidate['experience_years']} anos)")

## 👥 Parte 5: Banco de Dados de Candidatos

Dados mockados de candidatos baseados na estrutura do app WorkTree.

In [ ]:
def create_compatibility_prompt(candidate: Dict, job: Dict) -> str:
    """
    Cria um prompt otimizado para análise de compatibilidade usando Prompt Engineering.
    
    Args:
        candidate (Dict): Dados do candidato
        job (Dict): Dados da vaga
    
    Returns:
        str: Prompt estruturado para o Gemini
    """
    
    prompt = f"""
Você é um especialista em Recursos Humanos e Recrutamento Técnico com 15 anos de experiência 
em análise de compatibilidade entre candidatos e vagas de tecnologia.

**CONTEXTO DA PLATAFORMA:**
Esta análise será exibida no aplicativo mobile "WorkTree", uma plataforma de recrutamento 
inteligente. O candidato verá esta análise ao acessar os detalhes da vaga.

**TAREFA:**
Analise a compatibilidade entre o candidato e a vaga abaixo, fornecendo uma avaliação 
completa, objetiva e construtiva.

---

**DADOS DA VAGA:**

**Cargo:** {job['title']}
**Empresa:** {job['company']}
**Localização:** {job['location']}
**Tipo:** {job['type']}
**Categoria:** {job['category']}
**Salário:** {job['salary']}

**Descrição:**
{job['description']}

**Requisitos Obrigatórios:**
{chr(10).join(f'• {req}' for req in job['requirements'])}

**Habilidades Técnicas Requeridas:**
{', '.join(job['required_skills'])}

**Diferenciais (Nice to Have):**
{', '.join(job['nice_to_have'])}

**Responsabilidades:**
{chr(10).join(f'• {resp}' for resp in job['responsibilities'])}

---

**DADOS DO CANDIDATO:**

**Nome:** {candidate['name']}
**Cargo Atual:** {candidate['title']}
**Experiência:** {candidate['experience_years']} anos
**Formação:** {candidate['education']}

**Bio:**
{candidate['bio']}

**Habilidades Técnicas:**
{', '.join(candidate['skills'])}

**Certificações:**
{chr(10).join(f'• {cert}' for cert in candidate['certifications'])}

**Idiomas:**
{', '.join(candidate['languages'])}

**Experiências Anteriores:**
{chr(10).join(f"• {role['title']} na {role['company']} ({role['duration']}): {role['description']}" for role in candidate['previous_roles'])}

---

**FORMATO DA RESPOSTA (OBRIGATÓRIO):**

Retorne a análise EXATAMENTE no seguinte formato JSON (sem markdown, sem ```):

{{
  "compatibility_score": <número de 0 a 100>,
  "compatibility_level": "<Alto|Médio|Baixo>",
  "summary": "<resumo executivo de 2-3 linhas sobre o match>",
  "matching_skills": [
    "<habilidade1>", "<habilidade2>", ...
  ],
  "missing_skills": [
    "<habilidade_faltante1>", "<habilidade_faltante2>", ...
  ],
  "strengths": [
    "<ponto forte 1>",
    "<ponto forte 2>",
    "<ponto forte 3>"
  ],
  "areas_for_improvement": [
    "<área de melhoria 1>",
    "<área de melhoria 2>",
    "<área de melhoria 3>"
  ],
  "recommendations": [
    "<recomendação específica 1>",
    "<recomendação específica 2>",
    "<recomendação específica 3>"
  ],
  "experience_match": {{
    "required_years": <anos_requeridos_estimados>,
    "candidate_years": {candidate['experience_years']},
    "analysis": "<análise da experiência>"
  }},
  "salary_expectation": {{
    "job_range": "{job['salary']}",
    "alignment": "<Alinhado|Acima|Abaixo>",
    "comment": "<comentário sobre expectativa salarial>"
  }},
  "next_steps": "<sugestão clara do próximo passo para o candidato>"
}}

**CRITÉRIOS DE AVALIAÇÃO:**

1. **Compatibility Score (0-100):**
   - 90-100: Match excepcional, candidato ideal
   - 75-89: Alto match, candidato muito qualificado
   - 60-74: Match moderado, candidato qualificado com gaps
   - 40-59: Match baixo, gaps significativos
   - 0-39: Incompatibilidade alta

2. **Análise de Habilidades:**
   - Compare TODAS as habilidades requeridas com as do candidato
   - Considere sinônimos técnicos (ex: JS = JavaScript)
   - Identifique habilidades extras que agregam valor

3. **Experiência:**
   - Compare anos de experiência
   - Analise relevância das experiências anteriores
   - Considere progressão de carreira

4. **Fit Cultural e Tipo de Vaga:**
   - Considere localização (remoto/presencial/híbrido)
   - Tipo de contrato (CLT/PJ)
   - Nível da vaga vs. nível do candidato

5. **Recomendações:**
   - Seja ESPECÍFICO e ACIONÁVEL
   - Sugira cursos, certificações ou habilidades concretas
   - Seja sempre construtivo e motivador

Analise agora e retorne APENAS o JSON, sem texto adicional.
"""
    
    return prompt

print("Função de Prompt Engineering criada!")
print("Esta função estrutura prompts otimizados para análise precisa pelo Gemini")

In [ ]:
def analyze_compatibility(candidate: Dict, job: Dict, verbose: bool = True) -> Dict:
    """
    Analisa a compatibilidade entre candidato e vaga usando Google Gemini AI.
    
    Args:
        candidate (Dict): Dados do candidato
        job (Dict): Dados da vaga
        verbose (bool): Se True, exibe informações detalhadas do processamento
    
    Returns:
        Dict: Análise completa de compatibilidade em formato estruturado
    """
    
    try:
        if verbose:
            print(f"Analisando compatibilidade...")
            print(f"   Candidato: {candidate['name']}")
            print(f"   Vaga: {job['title']} - {job['company']}")
            print(f"   Aguarde, consultando Gemini AI...\n")
        
        # Criar prompt otimizado
        prompt = create_compatibility_prompt(candidate, job)
        
        # Enviar para o Gemini
        response = model.generate_content(prompt)
        
        # Extrair texto da resposta
        response_text = response.text.strip()
        
        # Limpar possíveis marcadores de código
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.startswith("```"):
            response_text = response_text[3:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]
        
        response_text = response_text.strip()
        
        # Parse JSON
        analysis = json.loads(response_text)
        
        # Adicionar metadados
        analysis['metadata'] = {
            'candidate_id': candidate['id'],
            'candidate_name': candidate['name'],
            'job_id': job['id'],
            'job_title': job['title'],
            'company': job['company'],
            'analyzed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'model_used': 'gemini-1.5-flash'
        }
        
        if verbose:
            print(f"Análise concluída com sucesso!")
            print(f"   Score de Compatibilidade: {analysis['compatibility_score']}%")
            print(f"   Nível: {analysis['compatibility_level']}\n")
        
        return analysis
        
    except json.JSONDecodeError as e:
        print(f"Erro ao fazer parse do JSON: {e}")
        print(f"Resposta recebida: {response_text[:500]}...")
        return None
        
    except Exception as e:
        print(f"Erro na análise: {e}")
        return None

print("Função de análise de compatibilidade criada!")
print("Esta função utiliza o Gemini para gerar análises inteligentes")

In [ ]:
def display_analysis(analysis: Dict):
    """
    Exibe a análise de compatibilidade de forma formatada e visualmente atraente.
    
    Args:
        analysis (Dict): Resultado da análise de compatibilidade
    """
    
    if not analysis:
        print("❌ Nenhuma análise disponível para exibir")
        return
    
    # Cabeçalho
    print("=" * 80)
    print("🎯 ANÁLISE DE COMPATIBILIDADE - WORKTREE".center(80))
    print("=" * 80)
    print()
    
    # Metadados
    meta = analysis['metadata']
    print(f"👤 Candidato: {meta['candidate_name']}")
    print(f"💼 Vaga: {meta['job_title']}")
    print(f"🏢 Empresa: {meta['company']}")
    print(f"📅 Analisado em: {meta['analyzed_at']}")
    print(f"🤖 Modelo: {meta['model_used']}")
    print()
    
    # Score de Compatibilidade
    score = analysis['compatibility_score']
    level = analysis['compatibility_level']
    
    # Emoji baseado no score
    if score >= 80:
        emoji = "🟢"
    elif score >= 60:
        emoji = "🟡"
    else:
        emoji = "🔴"
    
    print("-" * 80)
    print(f"{emoji} COMPATIBILIDADE: {score}% - Nível {level}".center(80))
    print("-" * 80)
    print()
    
    # Resumo
    print("📝 RESUMO:")
    print(f"   {analysis['summary']}")
    print()
    
    # Habilidades Compatíveis
    print("✅ HABILIDADES COMPATÍVEIS:")
    for skill in analysis['matching_skills']:
        print(f"   ✓ {skill}")
    print()
    
    # Habilidades Faltantes
    if analysis['missing_skills']:
        print("❌ HABILIDADES FALTANTES:")
        for skill in analysis['missing_skills']:
            print(f"   ✗ {skill}")
        print()
    
    # Pontos Fortes
    print("💪 PONTOS FORTES:")
    for i, strength in enumerate(analysis['strengths'], 1):
        print(f"   {i}. {strength}")
    print()
    
    # Áreas de Melhoria
    print("📈 ÁREAS PARA DESENVOLVIMENTO:")
    for i, area in enumerate(analysis['areas_for_improvement'], 1):
        print(f"   {i}. {area}")
    print()
    
    # Análise de Experiência
    exp = analysis['experience_match']
    print("🎓 ANÁLISE DE EXPERIÊNCIA:")
    print(f"   Requerido: ~{exp['required_years']} anos")
    print(f"   Candidato: {exp['candidate_years']} anos")
    print(f"   Avaliação: {exp['analysis']}")
    print()
    
    # Expectativa Salarial
    salary = analysis['salary_expectation']
    print("💰 EXPECTATIVA SALARIAL:")
    print(f"   Faixa da vaga: {salary['job_range']}")
    print(f"   Alinhamento: {salary['alignment']}")
    print(f"   Comentário: {salary['comment']}")
    print()
    
    # Recomendações
    print("🎯 RECOMENDAÇÕES:")
    for i, rec in enumerate(analysis['recommendations'], 1):
        print(f"   {i}. {rec}")
    print()
    
    # Próximos Passos
    print("👉 PRÓXIMOS PASSOS:")
    print(f"   {analysis['next_steps']}")
    print()
    
    print("=" * 80)

print("✅ Função de visualização criada!")
print("📊 Exibe resultados de forma clara e profissional")

In [ ]:
# Código da API REST Flask (para integração com o app mobile)
# Este código serve apenas como REFERÊNCIA
# A API real está no arquivo app.py na raiz do projeto

api_code = '''
from flask import Flask, request, jsonify
from flask_cors import CORS
import google.generativeai as genai
import json
import os
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

app = Flask(__name__)
CORS(app)  # Permitir requisições do app mobile

# Configurar Gemini API a partir de variável de ambiente
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

if not GOOGLE_API_KEY:
    print("⚠️ ATENÇÃO: GOOGLE_API_KEY não configurada!")
    print("Configure a variável de ambiente antes de usar a API")

model = genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',  # CORRIGIDO
    generation_config={
        'temperature': 0.7,
        'top_p': 0.95,
        'top_k': 40,
        'max_output_tokens': 2048,
    }
)

# ThreadPoolExecutor para processamento paralelo
executor = ThreadPoolExecutor(max_workers=10)

def create_compatibility_prompt(candidate, job):
    """Cria prompt para análise de compatibilidade"""
    # Implementação completa está no arquivo app.py
    # Esta é apenas uma referência
    pass

@app.route('/health', methods=['GET'])
def health_check():
    """Endpoint para verificar se a API está funcionando"""
    return jsonify({
        'status': 'online',
        'service': 'WorkTree AI Compatibility Analysis',
        'version': '1.0.0',
        'api_key_configured': GOOGLE_API_KEY is not None,
        'timestamp': datetime.now().isoformat()
    }), 200

@app.route('/api/analyze-compatibility', methods=['POST'])
def analyze_compatibility_endpoint():
    """
    Endpoint principal para análise de compatibilidade
    
    Espera JSON no formato:
    {
        "candidate": {
            "id": "c1",
            "name": "João Silva",
            "skills": ["React", "Node.js", ...],
            "experience_years": 5,
            ...
        },
        "job": {
            "id": "j1",
            "title": "Desenvolvedor Full Stack",
            "required_skills": ["React", "Node.js", ...],
            ...
        }
    }
    """
    try:
        data = request.get_json()
        
        if not data or 'candidate' not in data or 'job' not in data:
            return jsonify({
                'error': 'Missing required fields: candidate and job'
            }), 400
        
        candidate = data['candidate']
        job = data['job']
        
        # Validações básicas
        if not candidate.get('skills') or not job.get('required_skills'):
            return jsonify({
                'error': 'Missing required skills data'
            }), 400
        
        # Criar prompt
        prompt = create_compatibility_prompt(candidate, job)
        
        # Consultar Gemini
        response = model.generate_content(prompt)
        response_text = response.text.strip()
        
        # Limpar markdown
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.startswith("```"):
            response_text = response_text[3:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]
        
        # Parse JSON
        analysis = json.loads(response_text.strip())
        
        # Adicionar metadados
        analysis['metadata'] = {
            'candidate_id': candidate.get('id'),
            'job_id': job.get('id'),
            'analyzed_at': datetime.now().isoformat(),
            'model': 'gemini-1.5-flash'
        }
        
        return jsonify(analysis), 200
        
    except json.JSONDecodeError as e:
        return jsonify({
            'error': 'Failed to parse AI response',
            'details': str(e)
        }), 500
        
    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'details': str(e)
        }), 500

@app.route('/api/batch-analyze', methods=['POST'])
def batch_analyze():
    """
    Analisa múltiplos candidatos para uma vaga em paralelo
    
    Espera JSON:
    {
        "job": {...},
        "candidates": [{...}, {...}, ...]
    }
    
    Retorna candidatos ordenados por score (maior para menor)
    """
    try:
        data = request.get_json()
        
        if not data or 'job' not in data or 'candidates' not in data:
            return jsonify({
                'error': 'Missing required fields: job and candidates'
            }), 400
        
        job = data['job']
        candidates = data['candidates']
        
        # Limitar número de candidatos
        if len(candidates) > 50:
            return jsonify({
                'error': 'Maximum 50 candidates per batch'
            }), 400
        
        results = []
        
        # Processar em paralelo usando ThreadPoolExecutor
        def analyze_single(candidate):
            prompt = create_compatibility_prompt(candidate, job)
            response = model.generate_content(prompt)
            
            response_text = response.text.strip()
            if response_text.startswith("```json"):
                response_text = response_text[7:]
            if response_text.startswith("```"):
                response_text = response_text[3:]
            if response_text.endswith("```"):
                response_text = response_text[:-3]
            
            analysis = json.loads(response_text.strip())
            analysis['candidate_id'] = candidate.get('id')
            analysis['candidate_name'] = candidate.get('name')
            return analysis
        
        # Executar análises em paralelo
        futures = [executor.submit(analyze_single, c) for c in candidates]
        results = [f.result() for f in futures]
        
        # Ordenar por score (maior primeiro)
        results.sort(key=lambda x: x.get('compatibility_score', 0), reverse=True)
        
        return jsonify({
            'job_id': job.get('id'),
            'job_title': job.get('title'),
            'total_candidates': len(results),
            'analyzed_at': datetime.now().isoformat(),
            'results': results
        }), 200
        
    except Exception as e:
        return jsonify({
            'error': 'Batch analysis failed',
            'details': str(e)
        }), 500

if __name__ == '__main__':
    print("🚀 WorkTree AI API iniciada!")
    print("📡 Endpoints disponíveis:")
    print("   GET  /health")
    print("   POST /api/analyze-compatibility")
    print("   POST /api/batch-analyze")
    app.run(host='0.0.0.0', port=5000, debug=True)
'''

print("✅ Código da API Flask (REFERÊNCIA)")
print("📄 Este é apenas um exemplo - a API real está em app.py")
print("\n⚠️  IMPORTANTE:")
print("   - NUNCA exponha sua API Key no código")
print("   - Use variáveis de ambiente (GOOGLE_API_KEY)")
print("   - No Render: configure em Environment Variables")
print("\n📝 Para usar a API de produção:")
print("   1. Acesse: https://ia-8xoy.onrender.com")
print("   2. Endpoints:")
print("      GET  /health - Verificar status")
print("      POST /api/analyze-compatibility - Análise individual")
print("      POST /api/batch-analyze - Análise em lote (paralela)")